# 🔬 HYBRID PROPHET + LSTM AUTOMATED PIPELINE
## Multi-Resolution Hybrid Forecasting with Auto-Benchmark

### 🚀 AUTO-RUN CONFIGURATION:
**Just click "Run All" and the notebook will automatically:**
1. Train Hybrid models for ALL resolutions (1min, 5min, 15min)
2. Forecast BOTH targets (request_count, total_bytes)
3. Combine Prophet (trend + seasonality) with LSTM (residual patterns)
4. Detect anomalies using dynamic thresholds
5. Generate comprehensive benchmarks
6. Compare Hybrid vs Pure Prophet vs Pure LSTM

### 📊 Total Configurations:
- **3 resolutions** × **2 targets** = **6 hybrid models**
- Each configuration trains 3 models: Prophet, LSTM, Hybrid
- Expected runtime: **40-70 minutes** (with GPU)

### 🧬 Hybrid Architecture:
```
Prophet (Trend + Seasonality)
    ↓
Residual = Actual - Prophet_Forecast
    ↓
LSTM (Learn Residual Patterns)
    ↓
Final = Prophet_Forecast + LSTM_Residual
```

### 📁 Output Structure:
```
RESULTS_HYBRID/
├── 1min_request_count/
│   ├── prophet_model/ (forecast, components)
│   ├── lstm_model.keras
│   ├── scaler.pkl
│   ├── hybrid_predictions.csv
│   ├── anomalies.csv
│   ├── metrics_comparison.csv (Prophet vs LSTM vs Hybrid)
│   └── visualizations.png
├── ... (5 more configurations)
└── FINAL_BENCHMARK/
    ├── comprehensive_comparison.csv
    ├── hybrid_vs_individual.csv
    ├── final_report.txt
    └── benchmark_visualizations.png
```

In [61]:
# ===========================
# CELL 2: SETUP & INSTALLATIONS
# ===========================

!pip install prophet tensorflow scikit-learn -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.layers import Attention, Input, Concatenate
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
from datetime import datetime
import time
from typing import Dict, List, Tuple

# Prophet
from prophet import Prophet

# TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)

# Visualization
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (18, 6)

print("="*70)
print("HYBRID PROPHET + LSTM AUTOMATED TRAINING PIPELINE")
print("="*70)
print(f"  TensorFlow version: {tf.__version__}")
print(f"  GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")
print(f"  Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*70)

HYBRID PROPHET + LSTM AUTOMATED TRAINING PIPELINE
  TensorFlow version: 2.19.0
  GPU available: True
  Start time: 2026-02-02 13:14:03


In [ ]:
# ===========================
# CELL 3: GLOBAL CONFIGURATION
# ===========================

# Paths
DATA_DIR = 'data'
RESULTS_BASE_DIR = 'models/results_hybrid'

# Create base results directory
os.makedirs(RESULTS_BASE_DIR, exist_ok=True)

# All configurations to run
RESOLUTIONS = ['5min', '15min']
TARGETS = ['request_count', 'total_bytes']

# Resolution-specific parameters
RESOLUTION_PARAMS = {
    '5min': {
        'window': 10,        
        'lstm_units': 50,
        'epochs': 50,
        'batch_size': 50
    },
    '15min': {
        'window': 5,          
        'lstm_units': 50,
        'epochs': 50,
        'batch_size': 50 
    }
}

# Prophet parameters
PROPHET_PARAMS = {
    'daily_seasonality': False,
    'weekly_seasonality': False,
    'yearly_seasonality': False,
    'changepoint_prior_scale': 5,
    'seasonality_prior_scale': 30,
    'seasonality_mode': 'multiplicative'
}

# Storm/outage holiday
STORM_HOLIDAY = pd.DataFrame({
    'holiday': 'storm_outage',
    'ds': pd.date_range(start='1995-08-01 14:52:01', end='1995-08-03 04:36:13', freq='h'),
    'lower_window': 0,
    'upper_window': 0,
})

print("\n📋 CONFIGURATION LOADED:")
print(f"  Resolutions: {RESOLUTIONS}")
print(f"  Targets: {TARGETS}")
print(f"  Total configurations: {len(RESOLUTIONS) * len(TARGETS)}")
print(f"\n  Data directory: {DATA_DIR}")
print(f"  Results directory: {RESULTS_BASE_DIR}")


📋 CONFIGURATION LOADED:
  Resolutions: ['5min', '15min']
  Targets: ['request_count', 'total_bytes']
  Total configurations: 4

  Data directory: /kaggle/input/nasa-1/PROCESSED_DATAFINAL
  Results directory: /kaggle/working/


In [63]:
# ===========================
# CELL 4: UTILITY FUNCTIONS
# ===========================

def prepare_prophet_data(df, target_col):
    """
    Prepare data for Prophet.
    """
    prophet_df = pd.DataFrame({
        'ds': df.index,
        'y': df[target_col]
    })
    
    # Add time-based features
    prophet_df['hour'] = prophet_df['ds'].dt.hour
    prophet_df['day_of_week'] = prophet_df['ds'].dt.dayofweek
    prophet_df['is_weekend'] = (prophet_df['day_of_week'] >= 5).astype(int)
    
    return prophet_df


def make_sequences(data, window):
    """
    Create sequences for LSTM.
    """
    X, y = [], []
    for i in range(window, len(data)):
        X.append(data[i-window:i])
        y.append(data[i])
    return np.array(X), np.array(y)

def create_multivariate_features(df, train_size, window):
    """
    Tạo bộ dữ liệu đa chiều cho LSTM:
    1. Residual (Scaled)
    2. Prophet Prediction (Scaled) - Để học độ lớn
    3. Hour Sin/Cos - Để học tính chu kỳ thời gian
    """
    # 1. Feature: Residual
    resid_values = df[['residual']].values
    scaler_resid = StandardScaler()
    # Chỉ fit trên tập train
    scaler_resid.fit(resid_values[:train_size]) 
    resid_scaled = scaler_resid.transform(resid_values)
    
    # 2. Feature: Context (Prophet Prediction)
    yhat_values = df[['yhat_prophet']].values
    scaler_context = StandardScaler()
    scaler_context.fit(yhat_values[:train_size])
    yhat_scaled = scaler_context.transform(yhat_values)
    
    # 3. Feature: Time Cyclical Encoding
    # Biến đổi giờ thành vòng tròn (23h gần 0h)
    hour_sin = np.sin(2 * np.pi * df.index.hour / 24).values.reshape(-1, 1)
    hour_cos = np.cos(2 * np.pi * df.index.hour / 24).values.reshape(-1, 1)
    
    # Gộp tất cả features: Shape (N, 4)
    # [Residual, Prophet_Hat, Hour_Sin, Hour_Cos]
    features_matrix = np.hstack([resid_scaled, yhat_scaled, hour_sin, hour_cos])
    
    return features_matrix, scaler_resid

def make_multivariate_sequences(features, targets, window):
    """
    X: Multivariate features (t-window ... t-1)
    y: Target residual (t)
    """
    X, y = [], []
    for i in range(window, len(features)):
        X.append(features[i-window:i, :]) # Lấy cửa sổ của TẤT CẢ features
        y.append(targets[i])             # Chỉ dự đoán residual (đã scale)
    return np.array(X), np.array(y)


def calculate_metrics(y_true, y_pred, model_name="Model"):
    """
    Calculate comprehensive metrics.
    """
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    
    # MAPE (avoid division by zero)
    mask = y_true != 0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if mask.sum() > 0 else 0
    
    # R²
    r2 = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_true.mean())**2))
    
    metrics = {
        'Model': model_name,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2
    }
    
    return metrics


def detect_anomalies(y_true, y_pred, window_size, threshold=3):
    """
    Detect anomalies using dynamic Z-score on prediction error.
    """
    error = np.abs(y_true - y_pred)
    
    # Rolling statistics
    rolling_mean = pd.Series(error).rolling(window_size, min_periods=1).mean()
    rolling_std = pd.Series(error).rolling(window_size, min_periods=1).std()
    
    # Z-score
    z_score = (error - rolling_mean) / (rolling_std + 1e-8)
    
    # Anomaly indices
    anomaly_mask = z_score > threshold
    anomaly_indices = np.where(anomaly_mask)[0]
    
    return anomaly_indices, z_score.values


print("✓ Utility functions defined")

✓ Utility functions defined


In [64]:
# # ===========================================================================================
# # CELL 5: MAIN HYBRID TRAINING FUNCTION (FINAL VERSION: LOG + STORM MASKING)
# # ===========================================================================================
# from sklearn.preprocessing import MinMaxScaler
# from keras.layers import Bidirectional, LSTM, Dense, Dropout
# from keras.models import Sequential
# from keras.losses import Huber
# from keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from prophet import Prophet
# import numpy as np
# import pandas as pd
# import os
# import time
# import pickle

# def train_hybrid_model(resolution, target, verbose=True):
#     """
#     Train Hybrid Prophet + LSTM model with:
#     1. Storm Masking (NaN for outage periods)
#     2. Input Log Transformation (Amplitude Correction)
#     3. Robust LSTM Architecture (Bidirectional + Huber Loss)
#     """
#     if verbose:
#         print(f"\n{'='*70}")
#         print(f"TRAINING: Hybrid (Log + Storm-Masked) | {resolution} | {target}")
#         print(f"{'='*70}")
    
#     start_time = time.time()
    
#     # --- CẤU HÌNH ---
#     # Lấy params từ dict global hoặc hardcode nếu cần
#     params = RESOLUTION_PARAMS[resolution]
#     window = params['window']
#     lstm_units = params['lstm_units']
#     epochs = params['epochs']
#     batch_size = params['batch_size']
    
#     results_dir = f"{RESULTS_BASE_DIR}/{resolution}_{target}_final"
#     os.makedirs(results_dir, exist_ok=True)
#     os.makedirs(f"{results_dir}/prophet_model", exist_ok=True)
    
#     try:
#         # ==================
#         # 1. LOAD DATA & STORM MASKING
#         # ==================
#         if verbose: print(f"[1/7] Loading data & Masking Storm...")
        
#         train_df = pd.read_csv(f"{DATA_DIR}/train_{resolution}.csv", index_col=0, parse_dates=True)
#         test_df = pd.read_csv(f"{DATA_DIR}/test_{resolution}.csv", index_col=0, parse_dates=True)
        
#         # Lưu giá trị thực gốc (Raw scale) để đánh giá sau cùng
#         y_test_raw_true = test_df[target].values

#         # --- [NEW] XỬ LÝ BÃO (Gán NaN) ---
#         # Bão: 14:52:01 01/08/1995 -> 04:36:13 03/08/1995
#         # Chỉ xử lý trên Train set vì bão nằm trong tháng 8
#         storm_start = pd.Timestamp("1995-08-01 14:52:01")
#         storm_end   = pd.Timestamp("1995-08-03 04:36:13")
        
#         mask = (train_df.index >= storm_start) & (train_df.index <= storm_end)
#         train_df.loc[mask, target] = None # Gán NaN để Prophet bỏ qua
        
#         if verbose: print(f"   -> Masked {mask.sum()} intervals as NaN due to storm.")

#         # --- LOG TRANSFORMATION ---
#         # np.log1p xử lý tốt NaN (vẫn giữ là NaN)
#         train_df[target] = np.log1p(train_df[target])
#         test_df[target] = np.log1p(test_df[target])
        
#         full_df = pd.concat([train_df, test_df]).sort_index()
#         train_size = len(train_df)
        
#         # ==================
#         # 2. TRAIN PROPHET (LOG SPACE)
#         # ==================
#         if verbose: print(f"\n[2/7] Training Prophet on Log Data...")
        
#         prophet_train = prepare_prophet_data(train_df, target)
        
#         prophet_model = Prophet(
#             changepoint_prior_scale=5.1, 
#             seasonality_prior_scale=30,
#             seasonality_mode='additive', # Log space -> Additive
#         )
        
#         # Seasonality (Khớp bài báo)
#         prophet_model.add_seasonality(name='daily_high_freq', period=1, fourier_order=50)
#         prophet_model.add_seasonality(name='weekly_high_freq', period=7, fourier_order=20)
        
#         prophet_model.add_regressor('hour')
#         prophet_model.add_regressor('day_of_week')
#         prophet_model.add_regressor('is_weekend')
        
#         prophet_model.fit(prophet_train)
        
#         # Forecast
#         prophet_full = prepare_prophet_data(full_df, target)
#         prophet_forecast = prophet_model.predict(prophet_full[['ds', 'hour', 'day_of_week', 'is_weekend']])
        
#         # ==================
#         # 3. COMPUTE RESIDUALS
#         # ==================
#         full_df['yhat_prophet'] = prophet_forecast['yhat'].values
#         full_df['residual'] = full_df[target] - full_df['yhat_prophet']
        
#         # --- [NEW] FILLNA FOR RESIDUALS ---
#         # Tại vùng bão, target=NaN nên residual=NaN. LSTM không hiểu NaN.
#         # Ta fill 0 (coi như Prophet dự đoán đúng xu hướng, không có lỗi dư thừa)
#         full_df['residual'] = full_df['residual'].fillna(0)
        
#         if verbose: print(f" Residual std (log-space): {full_df['residual'].std():.3f}")
        
#         # ==================
#         # 4. PREPARE LSTM DATA
#         # ==================
#         scaler = MinMaxScaler(feature_range=(0, 1))
        
#         residual_train_values = full_df.iloc[:train_size][['residual']].values
#         scaler.fit(residual_train_values)
#         residual_train_scaled = scaler.transform(residual_train_values)
        
#         X_train, y_train = make_sequences(residual_train_scaled.flatten(), window)
#         X_train = X_train.reshape(-1, window, 1)
        
#         # Split Valid
#         val_ratio = 0.2
#         val_size = int(len(X_train) * val_ratio)
#         X_train_sub = X_train[:-val_size]
#         y_train_sub = y_train[:-val_size]
#         X_val = X_train[-val_size:]
#         y_val = y_train[-val_size:]
        
#         # ==================
#         # 5. TRAIN LSTM (ROBUST ARCHITECTURE)
#         # ==================
#         if verbose: print(f"\n[5/7] Training Robust LSTM...")
        
#         lstm_model = Sequential([
#             Bidirectional(LSTM(lstm_units, return_sequences=True, input_shape=(window, 1))),
#             Bidirectional(LSTM(lstm_units, return_sequences=False)),
#             Dropout(0.2),
#             Dense(1)
#         ])
        
#         lstm_model.compile(optimizer='adam', loss=Huber(delta=1.0), metrics=['mae'])
        
#         callbacks = [
#             EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss', verbose=1),
#             ReduceLROnPlateau(patience=5, factor=0.5, monitor='val_loss', verbose=1)
#         ]
        
#         history = lstm_model.fit(
#             X_train_sub, y_train_sub,
#             validation_data=(X_val, y_val),
#             epochs=epochs,
#             batch_size=batch_size,
#             callbacks=callbacks,
#             verbose=1
#         )
        
#         # ==================
#         # 6. HYBRID PREDICTION
#         # ==================
#         if verbose: print(f"\n[6/7] Generating hybrid predictions...")
        
#         # Predict Residuals (All data)
#         residual_all_values = full_df[['residual']].values
#         residual_all_scaled = scaler.transform(residual_all_values)
        
#         X_all, _ = make_sequences(residual_all_scaled.flatten(), window)
#         X_all = X_all.reshape(-1, window, 1)
        
#         residual_pred_scaled = lstm_model.predict(X_all, verbose=0)
#         residual_pred_log = scaler.inverse_transform(residual_pred_scaled).flatten()
        
#         # Reconstruct Hybrid (Log Space)
#         prophet_pred_log = full_df['yhat_prophet'].iloc[window:].values
#         hybrid_pred_log = prophet_pred_log + residual_pred_log
        
#         # Inverse Transform (Log -> Real)
#         hybrid_pred_final = np.expm1(hybrid_pred_log)
#         prophet_pred_final = np.expm1(prophet_pred_log)
        
#         # ==================
#         # 7. EVALUATION
#         # ==================
#         if verbose: print(f"\n[7/7] Evaluating models...")

#         # Cắt dữ liệu Test
#         lstm_test_start_idx = train_size - window
        
#         hybrid_test_pred = hybrid_pred_final[lstm_test_start_idx:]
#         prophet_test_pred = prophet_pred_final[lstm_test_start_idx:]
#         residual_test_pred = residual_pred_log[lstm_test_start_idx:] 

#         # Lấy dữ liệu thực tế (Lấy từ Log Residuals để so sánh LSTM)
#         residual_test_true = full_df['residual'].iloc[train_size:].values
        
#         # Đồng bộ độ dài
#         min_len = min(len(y_test_raw_true), len(hybrid_test_pred), len(residual_test_true))
        
#         y_test_true = y_test_raw_true[:min_len]
#         hybrid_test_pred = hybrid_test_pred[:min_len]
#         prophet_test_pred = prophet_test_pred[:min_len]
#         residual_test_true = residual_test_true[:min_len]
#         residual_test_pred = residual_test_pred[:min_len]
        
#         # Metrics
#         prophet_metrics = calculate_metrics(y_test_true, prophet_test_pred, "Prophet")
#         hybrid_metrics = calculate_metrics(y_test_true, hybrid_test_pred, "Hybrid")
#         lstm_metrics = calculate_metrics(residual_test_true, residual_test_pred, "LSTM_Residuals")
        
#         residual_mape = np.mean(
#             np.abs((residual_test_true - residual_test_pred) / (np.abs(residual_test_true) + 1e-8))
#         ) * 100
        
#         if verbose:
#             print(f"\n>>> LSTM Residuals MAPE on test: {residual_mape:.2f}% <<<")
#             print(
#                 f" Prophet - MAE: {prophet_metrics['MAE']:.2f}, "
#                 f"MSE: {prophet_metrics['MSE']:.2f}, "
#                 f"MAPE: {prophet_metrics['MAPE']:.2f}%, "
#                 f"RMSE: {prophet_metrics['RMSE']:.2f}"
#             )
#             # print(
#             #     f" LSTM Residuals - MAE: {lstm_metrics['MAE']:.2f}, "
#             #     f"MSE: {lstm_metrics['MSE']:.2f}, "
#             #     f"MAPE: {lstm_metrics['MAPE']:.2f}%, "
#             #     f"RMSE: {lstm_metrics['RMSE']:.2f}"
#             # )
#             print(
#                 f" Hybrid - MAE: {hybrid_metrics['MAE']:.2f}, "
#                 f"MSE: {hybrid_metrics['MSE']:.2f}, "
#                 f"MAPE: {hybrid_metrics['MAPE']:.2f}%, "
#                 f"RMSE: {hybrid_metrics['RMSE']:.2f}"
#             )
        
#         # Save results (Giữ nguyên code save của bạn)
#         elapsed_time = time.time() - start_time
#         return {
#             'resolution': resolution,
#             'target': target,
#             'prophet_mae': prophet_metrics['MAE'],
#             'prophet_rmse': prophet_metrics['RMSE'],
#             'prophet_mse': prophet_metrics['MSE'],
#             'prophet_mape': prophet_metrics['MAPE'],
#             'lstm_mae': lstm_metrics['MAE'],
#             'lstm_rmse': lstm_metrics['RMSE'],
#             'lstm_mse': lstm_metrics['MSE'],
#             'lstm_mape': lstm_metrics['MAPE'],
#             'hybrid_mae': hybrid_metrics['MAE'],
#             'hybrid_rmse': hybrid_metrics['RMSE'],
#             'hybrid_mse': hybrid_metrics['MSE'],
#             'hybrid_mape': hybrid_metrics['MAPE'],
#             'hybrid_r2': hybrid_metrics['R2'],
#             'residual_mape_test': residual_mape,
#             'training_time_sec': elapsed_time,
#             'results_dir': results_dir
#         }

#     except Exception as e:
#         print(f"\n❌ ERROR: {str(e)}")
#         import traceback
#         traceback.print_exc()
#         return None

In [65]:
# # ===========================================================================================
# # CELL 5: MAIN HYBRID TRAINING FUNCTION (NO LOG TRANSFORM VERSION)
# # ===========================================================================================
# from sklearn.preprocessing import MinMaxScaler
# from keras.layers import Bidirectional, LSTM, Dense, Dropout
# from keras.models import Sequential
# from keras.losses import Huber
# from keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from prophet import Prophet
# import numpy as np
# import pandas as pd
# import os
# import time
# import pickle

# def train_hybrid_model(resolution, target, verbose=True):
#     """
#     Train Hybrid Prophet + LSTM model WITHOUT Log Transformation.
#     Running on RAW DATA.
#     """
#     if verbose:
#         print(f"\n{'='*70}")
#         print(f"TRAINING: Hybrid (RAW DATA + Storm-Masked) | {resolution} | {target}")
#         print(f"{'='*70}")
    
#     start_time = time.time()
    
#     # --- CẤU HÌNH ---
#     # Lấy params từ dict global hoặc hardcode
#     try:
#         params = RESOLUTION_PARAMS[resolution]
#     except:
#         # Fallback nếu chưa define dict
#         params = {'window': 10, 'lstm_units': 50, 'epochs': 50, 'batch_size': 16}
        
#     window = params['window']
#     lstm_units = params['lstm_units']
#     epochs = params['epochs']
#     batch_size = params['batch_size']
    
#     results_dir = f"{RESULTS_BASE_DIR}/{resolution}_{target}_raw_no_log"
#     os.makedirs(results_dir, exist_ok=True)
#     os.makedirs(f"{results_dir}/prophet_model", exist_ok=True)
    
#     try:
#         # ==================
#         # 1. LOAD DATA & STORM MASKING
#         # ==================
#         if verbose: print(f"[1/7] Loading data & Masking Storm (Raw Data)...")
        
#         train_df = pd.read_csv(f"{DATA_DIR}/train_{resolution}.csv", index_col=0, parse_dates=True)
#         test_df = pd.read_csv(f"{DATA_DIR}/test_{resolution}.csv", index_col=0, parse_dates=True)
        
#         # Lưu giá trị thực gốc để đánh giá
#         y_test_raw_true = test_df[target].values

#         # --- XỬ LÝ BÃO (Gán NaN) ---
#         storm_start = pd.Timestamp("1995-08-01 14:52:01")
#         storm_end   = pd.Timestamp("1995-08-03 04:36:13")
        
#         mask = (train_df.index >= storm_start) & (train_df.index <= storm_end)
#         train_df.loc[mask, target] = None # Gán NaN
        
#         if verbose: print(f"   -> Masked {mask.sum()} intervals as NaN due to storm.")

#         # --- [REMOVED] LOG TRANSFORMATION ---
        
#         full_df = pd.concat([train_df, test_df]).sort_index()
#         train_size = len(train_df)
        
#         # ==================
#         # 2. TRAIN PROPHET (RAW SPACE)
#         # ==================
#         if verbose: print(f"\n[2/7] Training Prophet on Raw Data...")
        
#         prophet_train = prepare_prophet_data(train_df, target)
        
#         prophet_model = Prophet(
#             changepoint_prior_scale=5, 
#             seasonality_prior_scale=30,
#             # Lưu ý: Với dữ liệu Raw biến động mạnh, 'multiplicative' thường tốt hơn 'additive'.
#             # Tuy nhiên để chạy an toàn với số 0, ta giữ 'additive' hoặc cần xử lý số 0 nếu dùng 'multiplicative'.
#             # Ở đây giữ nguyên 'additive' để so sánh công bằng với bản Log.
#             seasonality_mode='multiplicative', 
#         )
        
#         # Seasonality
#         prophet_model.add_seasonality(name='daily_high_freq', period=1, fourier_order=50)
#         prophet_model.add_seasonality(name='weekly_high_freq', period=7, fourier_order=20)
        
#         prophet_model.add_regressor('hour')
#         prophet_model.add_regressor('day_of_week')
#         prophet_model.add_regressor('is_weekend')
        
#         prophet_model.fit(prophet_train)
        
#         # Forecast
#         prophet_full = prepare_prophet_data(full_df, target)
#         prophet_forecast = prophet_model.predict(prophet_full[['ds', 'hour', 'day_of_week', 'is_weekend']])
        
#         # ==================
#         # 3. COMPUTE RESIDUALS (RAW SPACE)
#         # ==================
#         full_df['yhat_prophet'] = prophet_forecast['yhat'].values
        
#         # Residual = Actual - Predicted (Không phải Log - Log nữa)
#         full_df['residual'] = full_df[target] - full_df['yhat_prophet']
        
#         # Fill NaN residuals bằng 0 (cho vùng bão)
#         full_df['residual'] = full_df['residual'].fillna(0)
        
#         if verbose: 
#             print(f" Residual std (raw-space): {full_df['residual'].std():.3f}")
#             # In ra max residual để thấy sự khác biệt với bản Log
#             print(f" Max Residual: {full_df['residual'].max():.3f}")
        
#         # ==================
#         # 4. PREPARE LSTM DATA
#         # ==================
#         # MinMaxScaler cực kỳ quan trọng ở đây vì biên độ dữ liệu Raw rất lớn
#         scaler = MinMaxScaler(feature_range=(-1, 1))
        
#         residual_train_values = full_df.iloc[:train_size][['residual']].values
#         scaler.fit(residual_train_values)
#         residual_train_scaled = scaler.transform(residual_train_values)
        
#         X_train, y_train = make_sequences(residual_train_scaled.flatten(), window)
#         X_train = X_train.reshape(-1, window, 1)
        
#         # Split Valid
#         val_ratio = 0.1
#         val_size = int(len(X_train) * val_ratio)
#         X_train_sub = X_train[:-val_size]
#         y_train_sub = y_train[:-val_size]
#         X_val = X_train[-val_size:]
#         y_val = y_train[-val_size:]
        
#         # ==================
#         # 5. TRAIN LSTM
#         # ==================
#         if verbose: print(f"\n[5/7] Training Robust LSTM...")
        
#         lstm_model = Sequential([
#             Bidirectional(LSTM(lstm_units, return_sequences=True, input_shape=(window, 1))),
#             Bidirectional(LSTM(lstm_units - 30, return_sequences=False)),
#             Dropout(0.2),
#             Dense(1)
#         ])
        
#         lstm_model.compile(optimizer='adam', loss=Huber(delta=1.0), metrics=['mae'])
        
#         callbacks = [
#             EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss', verbose=1),
#             ReduceLROnPlateau(patience=5, factor=0.5, monitor='val_loss', verbose=1)
#         ]
        
#         history = lstm_model.fit(
#             X_train_sub, y_train_sub,
#             validation_data=(X_val, y_val),
#             epochs=epochs,
#             batch_size=batch_size,
#             callbacks=callbacks,
#             verbose=1
#         )
        
#         # ==================
#         # 6. HYBRID PREDICTION
#         # ==================
#         if verbose: print(f"\n[6/7] Generating hybrid predictions...")
        
#         # Predict Residuals (All data)
#         residual_all_values = full_df[['residual']].values
#         residual_all_scaled = scaler.transform(residual_all_values)
        
#         X_all, _ = make_sequences(residual_all_scaled.flatten(), window)
#         X_all = X_all.reshape(-1, window, 1)
        
#         residual_pred_scaled = lstm_model.predict(X_all, verbose=0)
#         # Đây là Residual dự báo ở dạng RAW (đã inverse scale)
#         residual_pred_raw = scaler.inverse_transform(residual_pred_scaled).flatten()
        
#         # Reconstruct Hybrid (Raw Space)
#         # Hybrid = Prophet + Residual
#         prophet_pred_raw = full_df['yhat_prophet'].iloc[window:].values
#         hybrid_pred_raw = prophet_pred_raw + residual_pred_raw
        
#         # --- [REMOVED] INVERSE TRANSFORM ---
#         # Không cần np.expm1 vì dữ liệu đã là raw
#         hybrid_pred_final = hybrid_pred_raw
#         prophet_pred_final = prophet_pred_raw
        
#         # ==================
#         # 7. EVALUATION
#         # ==================
#         if verbose: print(f"\n[7/7] Evaluating models...")

#         # Cắt dữ liệu Test
#         lstm_test_start_idx = train_size - window
        
#         hybrid_test_pred = hybrid_pred_final[lstm_test_start_idx:]
#         prophet_test_pred = prophet_pred_final[lstm_test_start_idx:]
#         residual_test_pred = residual_pred_raw[lstm_test_start_idx:] 

#         # Lấy Residual thực tế (Raw Space)
#         residual_test_true = full_df['residual'].iloc[train_size:].values
        
#         # Đồng bộ độ dài
#         min_len = min(len(y_test_raw_true), len(hybrid_test_pred), len(residual_test_true))
        
#         y_test_true = y_test_raw_true[:min_len]
#         hybrid_test_pred = hybrid_test_pred[:min_len]
#         prophet_test_pred = prophet_test_pred[:min_len]
#         residual_test_true = residual_test_true[:min_len]
#         residual_test_pred = residual_test_pred[:min_len]
        
#         # Metrics
#         prophet_metrics = calculate_metrics(y_test_true, prophet_test_pred, "Prophet")
#         hybrid_metrics = calculate_metrics(y_test_true, hybrid_test_pred, "Hybrid")
        
#         # Metrics cho Residuals (Raw Space)
#         lstm_metrics = calculate_metrics(residual_test_true, residual_test_pred, "LSTM_Residuals")
        
#         residual_mape = np.mean(
#             np.abs((residual_test_true - residual_test_pred) / (np.abs(residual_test_true) + 1e-8))
#         ) * 100
        
#         if verbose:
#             print(f"\n>>> RESULT (Resolution: {resolution}) - NO LOG TRANSFORM <<<")
#             print("-" * 60)
#             print(f" LSTM Residuals - MAPE: {residual_mape:.2f}% (Prediction of Raw Errors)")
#             print("-" * 60)
#             print(
#                 f" Prophet - MAE: {prophet_metrics['MAE']:.2f}, "
#                 f"MSE: {prophet_metrics['MSE']:.2f}, "
#                 f"MAPE: {prophet_metrics['MAPE']:.2f}%, "
#                 f"RMSE: {prophet_metrics['RMSE']:.2f}"
#             )
#             print(
#                 f" Hybrid  - MAE: {hybrid_metrics['MAE']:.2f}, "
#                 f"MSE: {hybrid_metrics['MSE']:.2f}, "
#                 f"MAPE: {hybrid_metrics['MAPE']:.2f}%, "
#                 f"RMSE: {hybrid_metrics['RMSE']:.2f}"
#             )
        
#         # Save results
#         elapsed_time = time.time() - start_time
#         return {
#             'resolution': resolution,
#             'target': target,
#             'prophet_mae': prophet_metrics['MAE'],
#             'prophet_rmse': prophet_metrics['RMSE'],
#             'prophet_mse': prophet_metrics['MSE'],
#             'prophet_mape': prophet_metrics['MAPE'],
#             'lstm_mae': lstm_metrics['MAE'],
#             'lstm_rmse': lstm_metrics['RMSE'],
#             'lstm_mse': lstm_metrics['MSE'],
#             'lstm_mape': lstm_metrics['MAPE'],
#             'hybrid_mae': hybrid_metrics['MAE'],
#             'hybrid_rmse': hybrid_metrics['RMSE'],
#             'hybrid_mse': hybrid_metrics['MSE'],
#             'hybrid_mape': hybrid_metrics['MAPE'],
#             'hybrid_r2': hybrid_metrics['R2'],
#             'residual_mape_test': residual_mape,
#             'training_time_sec': elapsed_time,
#             'results_dir': results_dir
#         }

#     except Exception as e:
#         print(f"\n❌ ERROR: {str(e)}")
#         import traceback
#         traceback.print_exc()
#         return None

In [66]:
# # ===========================================================================================
# # CELL 5: MAIN HYBRID TRAINING FUNCTION (UPGRADE: SETUP_1 ARCHITECTURE + STORM MASKING)
# # ===========================================================================================
# from sklearn.preprocessing import MinMaxScaler
# from keras.layers import Bidirectional, LSTM, Dense, Dropout
# from keras.models import Sequential
# from keras.losses import Huber
# from keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from prophet import Prophet
# import numpy as np
# import pandas as pd
# import os
# import time
# import pickle

# def train_hybrid_model(resolution, target, verbose=True):
#     """
#     UPGRADED HYBRID MODEL:
#     - Data: Log Transformed + Storm Masking (Best Data Practice).
#     - Model: Setup_1 Architecture (Scaler -1to1, Bottleneck LSTM).
#     - Goal: Optimized for Autoscaling (Robust short-term forecasting).
#     """
#     if verbose:
#         print(f"\n{'='*70}")
#         print(f"TRAINING: Hybrid Upgrade (Setup_1 Style) | {resolution} | {target}")
#         print(f"{'='*70}")
    
#     start_time = time.time()
    
#     # --- CẤU HÌNH ---
#     # Lấy params (nếu chưa có dict thì dùng default)
#     try:
#         params = RESOLUTION_PARAMS[resolution]
#     except:
#         params = {'window': 12, 'lstm_units': 50, 'epochs': 50, 'batch_size': 16}
        
#     window = params['window']
#     lstm_units = params['lstm_units'] # Layer 1
#     lstm_units_2 = 20                 # Layer 2 (Bottleneck từ Setup_1)
#     epochs = params['epochs']
#     batch_size = params['batch_size']
    
#     results_dir = f"{RESULTS_BASE_DIR}/{resolution}_{target}_upgrade"
#     os.makedirs(results_dir, exist_ok=True)
#     os.makedirs(f"{results_dir}/prophet_model", exist_ok=True)
    
#     try:
#         # ==================
#         # 1. LOAD DATA & STORM MASKING (Vẫn giữ để tránh học rác từ bão)
#         # ==================
#         if verbose: print(f"[1/7] Loading data & Masking Storm...")
        
#         train_df = pd.read_csv(f"{DATA_DIR}/train_{resolution}.csv", index_col=0, parse_dates=True)
#         test_df = pd.read_csv(f"{DATA_DIR}/test_{resolution}.csv", index_col=0, parse_dates=True)
        
#         y_test_raw_true = test_df[target].values

#         # Xử lý bão (Gán NaN)
#         storm_start = pd.Timestamp("1995-08-01 14:52:01")
#         storm_end   = pd.Timestamp("1995-08-03 04:36:13")
#         mask = (train_df.index >= storm_start) & (train_df.index <= storm_end)
#         train_df.loc[mask, target] = None 
        
#         # Log Transform (Giữ lại vì Autoscaling cần xử lý spikes mượt hơn)
#         train_df[target] = np.log1p(train_df[target])
#         test_df[target] = np.log1p(test_df[target])
        
#         full_df = pd.concat([train_df, test_df]).sort_index()
#         train_size = len(train_df)
        
#         # ==================
#         # 2. TRAIN PROPHET
#         # ==================
#         if verbose: print(f"\n[2/7] Training Prophet...")
        
#         prophet_train = prepare_prophet_data(train_df, target)
        
#         prophet_model = Prophet(
#             changepoint_prior_scale=5.1, 
#             seasonality_prior_scale=30,
#             seasonality_mode='multiplicative',
#         )
#         prophet_model.add_seasonality(name='daily_high_freq', period=1, fourier_order=50)
#         prophet_model.add_seasonality(name='weekly_high_freq', period=7, fourier_order=20)
#         prophet_model.add_regressor('hour')
#         prophet_model.add_regressor('day_of_week')
#         prophet_model.add_regressor('is_weekend')
        
#         prophet_model.fit(prophet_train)
        
#         prophet_full = prepare_prophet_data(full_df, target)
#         prophet_forecast = prophet_model.predict(prophet_full[['ds', 'hour', 'day_of_week', 'is_weekend']])
        
#         # ==================
#         # 3. COMPUTE RESIDUALS & PREPARE LSTM
#         # ==================
#         full_df['yhat_prophet'] = prophet_forecast['yhat'].values
#         full_df['residual'] = full_df[target] - full_df['yhat_prophet']
#         full_df['residual'] = full_df['residual'].fillna(0) # Fill 0 cho vùng bão
        
#         # --- [UPGRADE TỪ SETUP_1]: Scaler (-1, 1) ---
#         # Tốt hơn cho residuals vì dữ liệu phân bố quanh 0
#         scaler = MinMaxScaler(feature_range=(-1, 1))
        
#         residual_train_values = full_df.iloc[:train_size][['residual']].values
#         scaler.fit(residual_train_values)
#         residual_train_scaled = scaler.transform(residual_train_values)
        
#         X_train, y_train = make_sequences(residual_train_scaled.flatten(), window)
#         X_train = X_train.reshape(-1, window, 1)
        
#         # Split Valid
#         val_ratio = 0.2
#         val_size = int(len(X_train) * val_ratio)
#         X_train_sub = X_train[:-val_size]
#         y_train_sub = y_train[:-val_size]
#         X_val = X_train[-val_size:]
#         y_val = y_train[-val_size:]
        
#         # ==================
#         # 5. TRAIN LSTM (UPGRADE: BOTTLENECK ARCHITECTURE)
#         # ==================
#         if verbose: print(f"\n[5/7] Training LSTM (Setup_1 Architecture: 50 -> 20)...")
        
#         # Kiến trúc ép mô hình học đặc trưng quan trọng (Setup_1)
#         lstm_model = Sequential([
#             Bidirectional(LSTM(lstm_units, return_sequences=True, input_shape=(window, 1))),
#             Dropout(0.2),
#             Bidirectional(LSTM(lstm_units_2, return_sequences=False)), # Giảm xuống 20 units
#             Dense(1)
#         ])
        
#         # Huber Loss vẫn là chân ái cho Autoscaling (chống nhiễu tốt hơn MSE)
#         lstm_model.compile(optimizer='adam', loss=Huber(delta=1.0), metrics=['mae'])
        
#         callbacks = [
#             EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss', verbose=1),
#             ReduceLROnPlateau(patience=5, factor=0.5, monitor='val_loss', verbose=1)
#         ]
        
#         history = lstm_model.fit(
#             X_train_sub, y_train_sub,
#             validation_data=(X_val, y_val),
#             epochs=epochs,
#             batch_size=batch_size,
#             callbacks=callbacks,
#             verbose=1
#         )
        
#         # ==================
#         # 6. HYBRID PREDICTION
#         # ==================
#         if verbose: print(f"\n[6/7] Generating predictions...")
        
#         # Predict Residuals (All data)
#         residual_all_values = full_df[['residual']].values
#         residual_all_scaled = scaler.transform(residual_all_values)
        
#         X_all, _ = make_sequences(residual_all_scaled.flatten(), window)
#         X_all = X_all.reshape(-1, window, 1)
        
#         residual_pred_scaled = lstm_model.predict(X_all, verbose=0)
#         residual_pred_log = scaler.inverse_transform(residual_pred_scaled).flatten()
        
#         # Reconstruct Hybrid (Log -> Exp)
#         prophet_pred_log = full_df['yhat_prophet'].iloc[window:].values
#         hybrid_pred_log = prophet_pred_log + residual_pred_log
        
#         hybrid_pred_final = np.expm1(hybrid_pred_log)
#         prophet_pred_final = np.expm1(prophet_pred_log)
        
#         # ==================
#         # 7. EVALUATION
#         # ==================
#         if verbose: print(f"\n[7/7] Evaluating...")

#         lstm_test_start_idx = train_size - window
        
#         hybrid_test_pred = hybrid_pred_final[lstm_test_start_idx:]
#         prophet_test_pred = prophet_pred_final[lstm_test_start_idx:]
#         residual_test_pred = residual_pred_log[lstm_test_start_idx:] 

#         residual_test_true = full_df['residual'].iloc[train_size:].values
        
#         min_len = min(len(y_test_raw_true), len(hybrid_test_pred), len(residual_test_true))
        
#         y_test_true = y_test_raw_true[:min_len]
#         hybrid_test_pred = hybrid_test_pred[:min_len]
#         prophet_test_pred = prophet_test_pred[:min_len]
#         residual_test_true = residual_test_true[:min_len]
#         residual_test_pred = residual_test_pred[:min_len]
        
#         # Metrics
#         prophet_metrics = calculate_metrics(y_test_true, prophet_test_pred, "Prophet")
#         hybrid_metrics = calculate_metrics(y_test_true, hybrid_test_pred, "Hybrid")
        
#         # Residual MAPE
#         residual_mape = np.mean(
#             np.abs((residual_test_true - residual_test_pred) / (np.abs(residual_test_true) + 1e-8))
#         ) * 100
        
#         if verbose:
#             print(f"\n>>> RESULT (Resolution: {resolution}) - UPGRADED <<<")
#             print("-" * 60)
#             print(f" LSTM Residuals - MAPE: {residual_mape:.2f}%")
#             print("-" * 60)
#             print(f" Prophet - MAE: {prophet_metrics['MAE']:.2f} | MAPE: {prophet_metrics['MAPE']:.2f}%")
#             print(f" Hybrid  - MAE: {hybrid_metrics['MAE']:.2f} | MAPE: {hybrid_metrics['MAPE']:.2f}%")
        
#         # Save results
#         elapsed_time = time.time() - start_time
#         return {
#             'resolution': resolution,
#             'target': target,
#             'hybrid_mae': hybrid_metrics['MAE'],
#             'hybrid_mape': hybrid_metrics['MAPE'],
#             'residual_mape': residual_mape,
#             'training_time_sec': elapsed_time
#         }

#     except Exception as e:
#         print(f"\n❌ ERROR: {str(e)}")
#         import traceback
#         traceback.print_exc()
#         return None

In [67]:
# # ===========================================================================================
# # CELL 5: SETUP_1 REVIVAL (RAW DATA + STORM MASKING + ONE-STEP AHEAD)
# # ===========================================================================================
# from sklearn.preprocessing import MinMaxScaler
# from keras.layers import Bidirectional, LSTM, Dense, Dropout
# from keras.models import Sequential
# from keras.losses import Huber
# from keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from prophet import Prophet
# import numpy as np
# import pandas as pd
# import os
# import time
# import pickle

# def train_hybrid_model(resolution, target, verbose=True):
#     if verbose:
#         print(f"\n{'='*70}")
#         print(f"TRAINING: Setup_1 Revival (RAW DATA + One-Step Ahead) | {resolution} | {target}")
#         print(f"{'='*70}")
    
#     start_time = time.time()
    
#     # --- CẤU HÌNH ---
#     try: params = RESOLUTION_PARAMS[resolution]
#     except: params = {'window': 12, 'lstm_units': 50, 'epochs': 50, 'batch_size': 16}
        
#     window = params['window']
#     lstm_units = params['lstm_units'] 
#     lstm_units_2 = 20  # Bottleneck từ Setup_1
#     epochs = params['epochs']
#     batch_size = params['batch_size']
    
#     results_dir = f"{RESULTS_BASE_DIR}/{resolution}_{target}_setup1_revival"
#     os.makedirs(results_dir, exist_ok=True)
#     os.makedirs(f"{results_dir}/prophet_model", exist_ok=True)
    
#     try:
#         # ==================
#         # 1. LOAD DATA & STORM MASKING
#         # ==================
#         if verbose: print(f"[1/7] Loading Data (RAW)...")
#         train_df = pd.read_csv(f"{DATA_DIR}/train_{resolution}.csv", index_col=0, parse_dates=True)
#         test_df = pd.read_csv(f"{DATA_DIR}/test_{resolution}.csv", index_col=0, parse_dates=True)
        
#         y_test_raw_true = test_df[target].values

#         # Masking Storm
#         storm_start = pd.Timestamp("1995-08-01 14:52:01")
#         storm_end   = pd.Timestamp("1995-08-03 04:36:13")
#         mask = (train_df.index >= storm_start) & (train_df.index <= storm_end)
#         train_df.loc[mask, target] = None 
        
#         # --- QUAN TRỌNG: KHÔNG DÙNG LOG TRANSFORM ---
#         # Chạy trực tiếp trên Raw Data
        
#         full_df = pd.concat([train_df, test_df]).sort_index()
#         train_size = len(train_df)
        
#         # ==================
#         # 2. TRAIN PROPHET
#         # ==================
#         if verbose: print(f"\n[2/7] Training Prophet (Raw Space)...")
#         prophet_train = prepare_prophet_data(train_df, target)
        
#         prophet_model = Prophet(
#             changepoint_prior_scale=5.1, 
#             seasonality_prior_scale=30,
#             seasonality_mode='multiplicative', # Raw data biến động mạnh nên thử multiplicative
#         )
#         prophet_model.add_seasonality(name='daily_high_freq', period=1, fourier_order=50)
#         prophet_model.add_seasonality(name='weekly_high_freq', period=7, fourier_order=20)
#         prophet_model.add_regressor('hour')
#         prophet_model.add_regressor('day_of_week')
#         prophet_model.add_regressor('is_weekend')
        
#         prophet_model.fit(prophet_train)
        
#         prophet_full = prepare_prophet_data(full_df, target)
#         prophet_forecast = prophet_model.predict(prophet_full[['ds', 'hour', 'day_of_week', 'is_weekend']])
        
#         # ==================
#         # 3. COMPUTE RESIDUALS
#         # ==================
#         full_df['yhat_prophet'] = prophet_forecast['yhat'].values
#         # Residual = Thực tế - Prophet
#         full_df['residual'] = full_df[target] - full_df['yhat_prophet']
#         full_df['residual'] = full_df['residual'].fillna(0) 
        
#         # --- SCALER (-1, 1) CỦA SETUP_1 ---
#         scaler = MinMaxScaler(feature_range=(-1, 1))
        
#         residual_train_values = full_df.iloc[:train_size][['residual']].values
#         scaler.fit(residual_train_values)
#         residual_train_scaled = scaler.transform(residual_train_values)
        
#         X_train, y_train = make_sequences(residual_train_scaled.flatten(), window)
#         X_train = X_train.reshape(-1, window, 1)
        
#         # Split Valid
#         val_size = int(len(X_train) * 0.2)
#         X_train_sub, y_train_sub = X_train[:-val_size], y_train[:-val_size]
#         X_val, y_val = X_train[-val_size:], y_train[-val_size:]
        
#         # ==================
#         # 5. TRAIN LSTM (SETUP_1 ARCHITECTURE)
#         # ==================
#         if verbose: print(f"\n[5/7] Training LSTM (Bottleneck 50->20)...")
        
#         lstm_model = Sequential([
#             LSTM(lstm_units, return_sequences=True, input_shape=(window, 1)),
#             Dropout(0.2),
#             LSTM(lstm_units, return_sequences=False), # Bottleneck
#             Dense(1)
#         ])
        
#         lstm_model.compile(optimizer='adam', loss='huber', metrics=['mae'])
        
#         lstm_model.fit(X_train_sub, y_train_sub, validation_data=(X_val, y_val), 
#                        epochs=epochs, batch_size=batch_size, verbose=1,
#                        callbacks=[EarlyStopping(patience=10, restore_best_weights=True)])
        
#         # ==================
#         # 6. HYBRID PREDICTION (ONE-STEP LOGIC)
#         # ==================
#         if verbose: print(f"\n[6/7] Generating Forecast (Raw Space)...")
        
#         # Tạo chuỗi input cho Test: Đuôi Train + Residual Test Thực Tế
#         residual_test_true = full_df['residual'].iloc[train_size:].values
#         residual_train_tail = full_df['residual'].iloc[train_size-window:train_size].values
        
#         # Nối lại
#         history_stream = np.concatenate([residual_train_tail, residual_test_true])
#         history_stream_scaled = scaler.transform(history_stream.reshape(-1, 1)).flatten()
        
#         # Tạo X_test dạng cuộn (Vectorized rolling window - Nhanh hơn vòng lặp for)
#         # Hàm make_sequences sẽ tự động tạo các cửa sổ trượt [t-window : t]
#         X_test_rolling, _ = make_sequences(history_stream_scaled, window)
#         X_test_rolling = X_test_rolling.reshape(-1, window, 1)
        
#         # Predict Residuals (Raw Scaled)
#         residual_test_pred_scaled = lstm_model.predict(X_test_rolling, verbose=0)
        
#         # Inverse Scale -> Raw Residuals
#         residual_test_pred_raw = scaler.inverse_transform(residual_test_pred_scaled).flatten()
        
#         # Cộng lại: Hybrid = Prophet (Raw) + LSTM Residual (Raw)
#         prophet_test_pred_raw = full_df['yhat_prophet'].iloc[train_size:].values
        
#         # Cắt khớp độ dài
#         min_len = min(len(prophet_test_pred_raw), len(residual_test_pred_raw))
#         hybrid_test_pred_final = prophet_test_pred_raw[:min_len] + residual_test_pred_raw[:min_len]
        
#         # Kẹp giá trị âm về 0 (nếu có)
#         hybrid_test_pred_final = np.maximum(hybrid_test_pred_final, 0)

#         # ==================
#         # 7. EVALUATION
#         # ==================
#         if verbose: print(f"\n[7/7] Evaluating...")

#         # Cắt dữ liệu thực tế và dự báo cho khớp nhau
#         y_test_true = y_test_raw_true[:min_len]
#         prophet_test_pred = prophet_test_pred_raw[:min_len]
#         hybrid_test_pred = hybrid_test_pred_final[:min_len]
        
#         # Tính metrics
#         prophet_metrics = calculate_metrics(y_test_true, prophet_test_pred, "Prophet")
#         hybrid_metrics = calculate_metrics(y_test_true, hybrid_test_pred, "Hybrid")
        
#         if verbose:
#             print(f"\n>>> RESULT (Resolution: {resolution}) - NO LOG TRANSFORM <<<")
#             print("-" * 60)
#             print(
#                 f" Prophet - MAE: {prophet_metrics['MAE']:.2f}, "
#                 f"MSE: {prophet_metrics['MSE']:.2f}, "
#                 f"MAPE: {prophet_metrics['MAPE']:.2f}%, "
#                 f"RMSE: {prophet_metrics['RMSE']:.2f}"
#             )
#             print(
#                 f" Hybrid  - MAE: {hybrid_metrics['MAE']:.2f}, "
#                 f"MSE: {hybrid_metrics['MSE']:.2f}, "
#                 f"MAPE: {hybrid_metrics['MAPE']:.2f}%, "
#                 f"RMSE: {hybrid_metrics['RMSE']:.2f}"
#             )
        
#         # Save results
#         # Lưu CSV để vẽ biểu đồ
#         test_timestamps = test_df.index[:min_len]
#         predictions_df = pd.DataFrame({
#             'timestamp': test_timestamps,
#             'actual': y_test_true,
#             'prophet_pred': prophet_test_pred,
#             'hybrid_pred': hybrid_test_pred
#         })
#         predictions_df.to_csv(f"{results_dir}/hybrid_predictions.csv", index=False)

#         elapsed_time = time.time() - start_time
#         return {
#             'resolution': resolution,
#             'target': target,
#             'prophet_mae': prophet_metrics['MAE'],
#             'prophet_rmse': prophet_metrics['RMSE'],
#             'prophet_mse': prophet_metrics['MSE'],
#             'prophet_mape': prophet_metrics['MAPE'],
#             'hybrid_mae': hybrid_metrics['MAE'],
#             'hybrid_rmse': hybrid_metrics['RMSE'],
#             'hybrid_mse': hybrid_metrics['MSE'],
#             'hybrid_mape': hybrid_metrics['MAPE'],
#             'hybrid_r2': hybrid_metrics['R2'],
#             'results_dir': results_dir
#         }

#     except Exception as e:
#         print(f"\n❌ ERROR: {str(e)}")
#         import traceback
#         traceback.print_exc()
#         return None

In [76]:
# ===========================================================================================
# CELL 5: SETUP_1 REPLICA (RAW DATA + STORM MASKING + EXACT LSTM ARCHITECTURE)
# ===========================================================================================
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Bidirectional, LSTM, Dense, Dropout
from keras.models import Sequential
from keras.losses import Huber
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from prophet import Prophet
import numpy as np
import pandas as pd
import os
import time
import pickle

def train_hybrid_model(resolution, target, verbose=True):
    if verbose:
        print(f"\n{'='*70}")
        print(f"TRAINING: Setup_1 Replica (Exact Match) | {resolution} | {target}")
        print(f"{'='*70}")
    
    start_time = time.time()
    
    # --- CẤU HÌNH ---
    try: params = RESOLUTION_PARAMS[resolution]
    except: params = {'window': 12, 'lstm_units': 50, 'epochs': 50, 'batch_size': 16}
        
    window = params['window'] # Chính là look_back trong code của bạn
    epochs = params['epochs']
    batch_size = params['batch_size']
    
    results_dir = f"{RESULTS_BASE_DIR}/{resolution}_{target}_setup1_replica"
    os.makedirs(results_dir, exist_ok=True)
    os.makedirs(f"{results_dir}/prophet_model", exist_ok=True)
    
    try:
        # ==================
        # 1. LOAD DATA & STORM MASKING
        # ==================
        if verbose: print(f"[1/7] Loading Data (RAW)...")
        train_df = pd.read_csv(f"{DATA_DIR}/train_{resolution}.csv", index_col=0, parse_dates=True)
        test_df = pd.read_csv(f"{DATA_DIR}/test_{resolution}.csv", index_col=0, parse_dates=True)
        
        y_test_raw_true = test_df[target].values

        # Masking Storm
        storm_start = pd.Timestamp("1995-08-01 14:52:01")
        storm_end   = pd.Timestamp("1995-08-03 04:36:13")
        mask = (train_df.index >= storm_start) & (train_df.index <= storm_end)
        train_df.loc[mask, target] = None 
        
        # KHÔNG DÙNG LOG (Raw Data)
        
        full_df = pd.concat([train_df, test_df]).sort_index()
        train_size = len(train_df)
        
        # ==================
        # 2. TRAIN PROPHET
        # ==================
        if verbose: print(f"\n[2/7] Training Prophet (Raw Space)...")
        prophet_train = prepare_prophet_data(train_df, target)
        
        prophet_model = Prophet(
            changepoint_prior_scale=5.1, 
            seasonality_prior_scale=30,
            seasonality_mode='multiplicative',
        )
        prophet_model.add_seasonality(name='daily_high_freq', period=1, fourier_order=50)
        prophet_model.add_seasonality(name='weekly_high_freq', period=7, fourier_order=20)
        prophet_model.add_regressor('hour')
        prophet_model.add_regressor('day_of_week')
        prophet_model.add_regressor('is_weekend')
        
        prophet_model.fit(prophet_train)
        
        prophet_full = prepare_prophet_data(full_df, target)
        prophet_forecast = prophet_model.predict(prophet_full[['ds', 'hour', 'day_of_week', 'is_weekend']])
        
        # ==================
        # 3. COMPUTE RESIDUALS
        # ==================
        full_df['yhat_prophet'] = prophet_forecast['yhat'].values
        full_df['residual'] = full_df[target] - full_df['yhat_prophet']
        full_df['residual'] = full_df['residual'].fillna(0) 
        
        # --- SCALER (-1, 1) CỦA SETUP_1 ---
        scaler = MinMaxScaler(feature_range=(-1, 1))
        
        residual_train_values = full_df.iloc[:train_size][['residual']].values
        scaler.fit(residual_train_values)
        residual_train_scaled = scaler.transform(residual_train_values)
        
        X_train, y_train = make_sequences(residual_train_scaled.flatten(), window)
        X_train = X_train.reshape(-1, window, 1)
        
        # Split Valid
        val_size = int(len(X_train) * 0.2)
        X_train_sub, y_train_sub = X_train[:-val_size], y_train[:-val_size]
        X_val, y_val = X_train[-val_size:], y_train[-val_size:]
        
        # ==================
        # 5. TRAIN LSTM (EXACT SETUP_1 SYNTAX)
        # ==================
        if verbose: print(f"\n[5/7] Training LSTM (Exact Setup_1 Code)...")
        
        # --- [CHỈNH SỬA] GIỐNG HỆT SETUP_1 ---
        lstm_model = Sequential()
        # Layer 1: 50 units
        lstm_model.add(Bidirectional(LSTM(50, return_sequences=True), input_shape=(window, 1)))
        lstm_model.add(Dropout(0.2))
        # Layer 2: 20 units (Bottleneck)
        lstm_model.add(Bidirectional(LSTM(20))) # Mặc định return_sequences=False
        lstm_model.add(Dense(1))
        
        # Compile: dùng string 'huber' và 'adam' cho giống Setup_1
        lstm_model.compile(loss='huber', optimizer='adam') 
        
        # Fit
        lstm_model.fit(X_train_sub, y_train_sub, validation_data=(X_val, y_val), 
                       epochs=epochs, batch_size=batch_size, verbose=1,
                       callbacks=[EarlyStopping(patience=10, restore_best_weights=True)])
        
        # ==================
        # 6. HYBRID PREDICTION (VARIABLE NAMES MATCH SETUP_1)
        # ==================
        if verbose: print(f"\n[6/7] Generating Forecast (Concatenate Style)...")
        
        # 1. Lấy dữ liệu
        residual_test_true = full_df['residual'].iloc[train_size:].values
        # Lấy đuôi train (chính là train_residuals[-look_back:] trong Setup_1)
        residual_train_tail = full_df['residual'].iloc[train_size-window:train_size].values
        
        # 2. Nối lại (GIỐNG BIẾN total_residuals)
        # total_residuals = np.concatenate((train_residuals[-look_back:], test_residuals_true))
        total_residuals = np.concatenate([residual_train_tail, residual_test_true])
        
        # 3. Scale (-1, 1)
        # total_residuals_scaled = resid_scaler.transform(...)
        total_residuals_scaled = scaler.transform(total_residuals.reshape(-1, 1)).flatten()
        
        # 4. Tạo X_test (Sliding Window)
        X_test_rolling, _ = make_sequences(total_residuals_scaled, window)
        X_test_rolling = X_test_rolling.reshape(-1, window, 1)
        
        # 5. Predict
        residual_test_pred_scaled = lstm_model.predict(X_test_rolling, verbose=0)
        
        # 6. Inverse Scale
        residual_test_pred_raw = scaler.inverse_transform(residual_test_pred_scaled).flatten()
        
        # 7. Cộng lại: Prophet + LSTM
        prophet_test_pred_raw = full_df['yhat_prophet'].iloc[train_size:].values
        
        # Cắt khớp
        min_len = min(len(prophet_test_pred_raw), len(residual_test_pred_raw))
        hybrid_test_pred_final = prophet_test_pred_raw[:min_len] + residual_test_pred_raw[:min_len]
        
        # Kẹp giá trị âm về 0 (Setup_1 có bước này: np.maximum(final_pred, 0))
        hybrid_test_pred_final = np.maximum(hybrid_test_pred_final, 0)

        # ==================
        # 7. EVALUATION
        # ==================
        if verbose: print(f"\n[7/7] Evaluating...")

        y_test_true = y_test_raw_true[:min_len]
        prophet_test_pred = prophet_test_pred_raw[:min_len]
        hybrid_test_pred = hybrid_test_pred_final[:min_len]
        
        # Metrics
        prophet_metrics = calculate_metrics(y_test_true, prophet_test_pred, "Prophet")
        hybrid_metrics = calculate_metrics(y_test_true, hybrid_test_pred, "Hybrid")
        
        if verbose:
            print(f"\n>>> RESULT (Resolution: {resolution}) - SETUP_1 REPLICA <<<")
            print("-" * 60)
            print(
                f" Prophet - MAE: {prophet_metrics['MAE']:.2f}, "
                f"MSE: {prophet_metrics['MSE']:.2f}, "
                f"MAPE: {prophet_metrics['MAPE']:.2f}%, "
                f"RMSE: {prophet_metrics['RMSE']:.2f}"
            )
            print(
                f" Hybrid  - MAE: {hybrid_metrics['MAE']:.2f}, "
                f"MSE: {hybrid_metrics['MSE']:.2f}, "
                f"MAPE: {hybrid_metrics['MAPE']:.2f}%, "
                f"RMSE: {hybrid_metrics['RMSE']:.2f}"
            )
        
        # Save results
        test_timestamps = test_df.index[:min_len]
        predictions_df = pd.DataFrame({
            'timestamp': test_timestamps,
            'actual': y_test_true,
            'prophet_pred': prophet_test_pred,
            'hybrid_pred': hybrid_test_pred
        })
        predictions_df.to_csv(f"{results_dir}/hybrid_predictions.csv", index=False)

        elapsed_time = time.time() - start_time
        return {
            'resolution': resolution,
            'target': target,
            'prophet_mae': prophet_metrics['MAE'],
            'prophet_rmse': prophet_metrics['RMSE'],
            'prophet_mse': prophet_metrics['MSE'],
            'prophet_mape': prophet_metrics['MAPE'],
             'prophet_r2': prophet_metrics['R2'],
            'hybrid_mae': hybrid_metrics['MAE'],
            'hybrid_rmse': hybrid_metrics['RMSE'],
            'hybrid_mse': hybrid_metrics['MSE'],
            'hybrid_mape': hybrid_metrics['MAPE'],
            'hybrid_r2': hybrid_metrics['R2'],
            'results_dir': results_dir
        }

    except Exception as e:
        print(f"\n❌ ERROR: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

In [77]:
# ===========================
# CELL 6: RUN ALL CONFIGURATIONS
# ===========================

print("\n" + "="*70)
print("STARTING AUTOMATED HYBRID TRAINING PIPELINE")
print("="*70)

all_results = []
total_configs = len(RESOLUTIONS) * len(TARGETS)
current_config = 0

pipeline_start_time = time.time()

for resolution in RESOLUTIONS:
    for target in TARGETS:
        current_config += 1
        
        print(f"\n\n{'#'*70}")
        print(f"CONFIGURATION {current_config}/{total_configs}")
        print(f"{'#'*70}")
        
        result = train_hybrid_model(resolution, target, verbose=True)
        
        if result is not None:
            all_results.append(result)
            print(f"\n✅ Configuration {current_config}/{total_configs} completed successfully")
        else:
            print(f"\n❌ Configuration {current_config}/{total_configs} failed")
        
        # Progress update
        elapsed = time.time() - pipeline_start_time
        avg_time = elapsed / current_config
        remaining = (total_configs - current_config) * avg_time
        
        print(f"\n📊 Progress: {current_config}/{total_configs} ({current_config/total_configs*100:.1f}%)")
        print(f"   Elapsed: {elapsed/60:.1f} min | Est. remaining: {remaining/60:.1f} min")

total_elapsed = time.time() - pipeline_start_time

print("\n" + "="*70)
print("ALL CONFIGURATIONS COMPLETED")
print("="*70)
print(f"  Total time: {total_elapsed/60:.1f} minutes")
print(f"  Successful: {len(all_results)}/{total_configs}")
print(f"  Failed: {total_configs - len(all_results)}")


STARTING AUTOMATED HYBRID TRAINING PIPELINE


######################################################################
CONFIGURATION 1/4
######################################################################

TRAINING: Setup_1 Replica (Exact Match) | 5min | request_count
[1/7] Loading Data (RAW)...

[2/7] Training Prophet (Raw Space)...


13:22:53 - cmdstanpy - INFO - Chain [1] start processing
13:23:27 - cmdstanpy - INFO - Chain [1] done processing



[5/7] Training LSTM (Exact Setup_1 Code)...
Epoch 1/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0049 - val_loss: 0.0020
Epoch 2/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0027 - val_loss: 0.0020
Epoch 3/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0026 - val_loss: 0.0020
Epoch 4/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0025 - val_loss: 0.0020
Epoch 5/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0025 - val_loss: 0.0019
Epoch 6/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0025 - val_loss: 0.0019
Epoch 7/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0025 - val_loss: 0.0019
Epoch 8/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0025 - val_loss: 0.0019
Epoch 9/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0025 - val_loss: 0.0019
Epoch 10/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0025 - val_loss: 0.0019
Epoch 11/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0025 - val_loss: 0.00

13:24:23 - cmdstanpy - INFO - Chain [1] start processing
13:24:47 - cmdstanpy - INFO - Chain [1] done processing



[5/7] Training LSTM (Exact Setup_1 Code)...
Epoch 1/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0056 - val_loss: 0.0039
Epoch 2/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0048 - val_loss: 0.0038
Epoch 3/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0047 - val_loss: 0.0037
Epoch 4/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0047 - val_loss: 0.0037
Epoch 5/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0047 - val_loss: 0.0037
Epoch 6/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0047 - val_loss: 0.0037
Epoch 7/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0047 - val_loss: 0.0037
Epoch 8/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0046 - val_loss: 0.0037
Epoch 9/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0046 - val_loss: 0.0037
Epoch 10/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0046 - val_loss: 0.0037
Epoch 11/50
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0046 - val_loss: 0.0037
Epo

13:25:38 - cmdstanpy - INFO - Chain [1] start processing
13:25:46 - cmdstanpy - INFO - Chain [1] done processing



[5/7] Training LSTM (Exact Setup_1 Code)...
Epoch 1/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 0.0249 - val_loss: 0.0017
Epoch 2/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0028 - val_loss: 0.0017
Epoch 3/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0028 - val_loss: 0.0017
Epoch 4/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0028 - val_loss: 0.0017
Epoch 5/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0027 - val_loss: 0.0017
Epoch 6/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0026 - val_loss: 0.0017
Epoch 7/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0026 - val_loss: 0.0017
Epoch 8/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0026 - val_loss: 0.0017
Epoch 9/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0026 - val_loss: 0.0017
Epoch 10/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0025 - val_loss: 0.0017
Epoch 11/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0025 - val_loss: 0.0017
Epoch 12/50
82/82 ━━━━━

13:26:32 - cmdstanpy - INFO - Chain [1] start processing
13:26:38 - cmdstanpy - INFO - Chain [1] done processing



[5/7] Training LSTM (Exact Setup_1 Code)...
Epoch 1/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0108 - val_loss: 0.0045
Epoch 2/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0052 - val_loss: 0.0045
Epoch 3/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0050 - val_loss: 0.0044
Epoch 4/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0050 - val_loss: 0.0044
Epoch 5/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0050 - val_loss: 0.0043
Epoch 6/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0050 - val_loss: 0.0043
Epoch 7/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0049 - val_loss: 0.0043
Epoch 8/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0049 - val_loss: 0.0043
Epoch 9/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0049 - val_loss: 0.0044
Epoch 10/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0049 - val_loss: 0.0043
Epoch 11/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0049 - val_loss: 0.0043
Epoch 12/50
82/82 

In [79]:
# ===========================
# CELL 7: CREATE COMPREHENSIVE BENCHMARK
# ===========================

print("\n" + "="*70)
print("GENERATING COMPREHENSIVE BENCHMARK")
print("="*70)

# Create benchmark directory
benchmark_dir = f"{RESULTS_BASE_DIR}/FINAL_BENCHMARK"
os.makedirs(benchmark_dir, exist_ok=True)

# Convert results to DataFrame
benchmark_df = pd.DataFrame(all_results)

# Save comprehensive comparison
benchmark_file = f"{benchmark_dir}/comprehensive_comparison.csv"
benchmark_df.to_csv(benchmark_file, index=False)
print(f"\n✓ Benchmark saved: {benchmark_file}")

print("\n📊 BENCHMARK RESULTS:\n")
display(benchmark_df.style.background_gradient(
    cmap='RdYlGn_r', 
    subset=['prophet_mae', 'hybrid_mae', 'hybrid_rmse']
).format({
    'prophet_mae': '{:.2f}',
    'prophet_rmse': '{:.2f}',
    'prophet_mse': '{:.2f}',
    'prophet_mape': '{:.2f}%',
    'prophet_r2': '{:.4f}',
    'hybrid_mae': '{:.2f}',
    'hybrid_rmse': '{:.2f}',
    'hybrid_mse': '{:.2f}',
    'hybrid_mape': '{:.2f}%',
    'hybrid_r2': '{:.4f}',
    'training_time_sec': '{:.1f}s'
}))

# Calculate improvements
print("\n" + "="*70)
print("HYBRID IMPROVEMENTS")
print("="*70)

for idx, row in benchmark_df.iterrows():
    prophet_imp = ((row['prophet_mae'] - row['hybrid_mae']) / row['prophet_mae']) * 100
    
    print(f"\n{row['resolution']} | {row['target']}:")
    print(f"  Hybrid MAE: {row['hybrid_mae']:.2f}")
    print(f"  Improvement over Prophet: {prophet_imp:+.1f}%")
    print(f"  Improvement over LSTM: {lstm_imp:+.1f}%")
    # print(f"  Anomaly rate: {row['anomaly_rate']:.2f}%")

# Overall statistics
print("\n" + "="*70)
print("OVERALL STATISTICS")
print("="*70)

avg_prophet_imp = ((benchmark_df['prophet_mae'] - benchmark_df['hybrid_mae']) / benchmark_df['prophet_mae']).mean() * 100

print(f"\nAverage Hybrid Improvement:")
print(f"  vs Prophet: {avg_prophet_imp:+.1f}%")
# print(f"\nAverage Anomaly Rate: {benchmark_df['anomaly_rate'].mean():.2f}%")
# print(f"Total Anomalies Detected: {benchmark_df['anomalies_detected'].sum():,}")


GENERATING COMPREHENSIVE BENCHMARK

✓ Benchmark saved: /kaggle/working//FINAL_BENCHMARK/comprehensive_comparison.csv

📊 BENCHMARK RESULTS:



,resolution,target,prophet_mae,prophet_rmse,prophet_mse,prophet_mape,prophet_r2,hybrid_mae,hybrid_rmse,hybrid_mse,hybrid_mape,hybrid_r2,results_dir
0,5min,request_count,57.08,81.34,6615.84,36.60%,0.5584,34.96,46.99,2208.31,27.71%,0.8526,/kaggle/working//5min_request_count_setup1_replica
1,5min,total_bytes,928196.66,1289904.19,1663852821668.35,38.37%,0.3530,632106.99,847233.17,717804046790.15,29.33%,0.7209,/kaggle/working//5min_total_bytes_setup1_replica
2,15min,request_count,154.81,226.28,51201.37,24.46%,0.5933,83.81,114.77,13171.50,16.03%,0.8954,/kaggle/working//15min_request_count_setup1_replica
3,15min,total_bytes,2491716.33,3485437.97,12148277865859.69,30.77%,0.3990,1502758.73,1998017.18,3992072664394.16,22.56%,0.8025,/kaggle/working//15min_total_bytes_setup1_replica



HYBRID IMPROVEMENTS

5min | request_count:
  Hybrid MAE: 34.96
  Improvement over Prophet: +38.8%
  Improvement over LSTM: +0.0%

5min | total_bytes:
  Hybrid MAE: 632106.99
  Improvement over Prophet: +31.9%
  Improvement over LSTM: +0.0%

15min | request_count:
  Hybrid MAE: 83.81
  Improvement over Prophet: +45.9%
  Improvement over LSTM: +0.0%

15min | total_bytes:
  Hybrid MAE: 1502758.73
  Improvement over Prophet: +39.7%
  Improvement over LSTM: +0.0%

OVERALL STATISTICS

Average Hybrid Improvement:
  vs Prophet: +39.1%


In [85]:
# ===========================
# CELL 10: SUMMARY & NEXT STEPS
# ===========================

print("\n" + "="*80)
print("🎉 AUTOMATED HYBRID PIPELINE COMPLETED SUCCESSFULLY!")
print("="*80)

print(f"\n📊 SUMMARY:")
print(f"  Total configurations: {len(all_results)}")
print(f"  Total time: {total_elapsed/60:.1f} minutes")
print(f"  Average time per config: {total_elapsed/len(all_results):.1f} seconds")

print(f"\n📁 RESULTS LOCATION:")
print(f"  Main directory: {RESULTS_BASE_DIR}")
print(f"  Benchmark: {benchmark_dir}")

print(f"\n🏆 BEST CONFIGURATION:")
print(f"  {best['resolution']} | {best['target']}")
print(f"  Hybrid MAE: {best['hybrid_mae']:.2f}")
print(f"  Improvement: {avg_prophet_imp:+.1f}% vs Prophet, {avg_lstm_imp:+.1f}% vs LSTM")

# print(f"\n🔍 ANOMALY DETECTION:")
# print(f"  Total anomalies: {benchmark_df['anomalies_detected'].sum():,}")
# print(f"  Average rate: {benchmark_df['anomaly_rate'].mean():.2f}%")

print(f"\n📈 TOP 3 PERFORMERS (by Hybrid MAE):")
top_3 = benchmark_df.nsmallest(3, 'hybrid_mae')[['resolution', 'target', 'hybrid_mae', 'hybrid_rmse']]
for idx, row in top_3.iterrows():
    print(f"  {row['resolution']:5s} | {row['target']:15s} | MAE: {row['hybrid_mae']:6.2f} | RMSE: {row['hybrid_rmse']:6.2f}")

print(f"\n💡 NEXT STEPS:")
print(f"  1. Review final_report.txt in {benchmark_dir}")
print(f"  2. Check benchmark_visualizations.png")
print(f"  3. Analyze anomalies.csv for each configuration")
print(f"  4. Deploy best hybrid model to production")
print(f"  5. Set up monitoring for anomaly alerts")
print(f"  6. Implement retraining pipeline (weekly Prophet, daily LSTM)")

print(f"\n" + "="*80)
print("All results have been saved to Google Drive!")
print("="*80)


🎉 AUTOMATED HYBRID PIPELINE COMPLETED SUCCESSFULLY!

📊 SUMMARY:
  Total configurations: 4
  Total time: 4.6 minutes
  Average time per config: 69.2 seconds

📁 RESULTS LOCATION:
  Main directory: /kaggle/working/
  Benchmark: /kaggle/working//FINAL_BENCHMARK

🏆 BEST CONFIGURATION:
  5min | request_count
  Hybrid MAE: 34.96
  Improvement: +39.1% vs Prophet, -0.0% vs LSTM

📈 TOP 3 PERFORMERS (by Hybrid MAE):
  5min  | request_count   | MAE:  34.96 | RMSE:  46.99
  15min | request_count   | MAE:  83.81 | RMSE: 114.77
  5min  | total_bytes     | MAE: 632106.99 | RMSE: 847233.17

💡 NEXT STEPS:
  1. Review final_report.txt in /kaggle/working//FINAL_BENCHMARK
  2. Check benchmark_visualizations.png
  3. Analyze anomalies.csv for each configuration
  4. Deploy best hybrid model to production
  5. Set up monitoring for anomaly alerts
  6. Implement retraining pipeline (weekly Prophet, daily LSTM)

All results have been saved to Google Drive!


In [86]:
# Nén toàn bộ file trong thư mục hiện tại (recursive)
!zip -r all_output.zip .

# Tạo link tải
from IPython.display import FileLink
FileLink('all_output.zip')

updating: 5min_request_count_raw_no_log/ (stored 0%)
updating: 5min_request_count_raw_no_log/prophet_model/ (stored 0%)
updating: 15min_request_count_raw_no_log/ (stored 0%)
updating: 15min_request_count_raw_no_log/prophet_model/ (stored 0%)
updating: 5min_request_count_final/ (stored 0%)
updating: 5min_request_count_final/prophet_model/ (stored 0%)
updating: .virtual_documents/ (stored 0%)
updating: .virtual_documents/__notebook_source__.ipynb (deflated 80%)
updating: 5min_total_bytes_raw_no_log/ (stored 0%)
updating: 5min_total_bytes_raw_no_log/prophet_model/ (stored 0%)
updating: 15min_total_bytes_raw_no_log/ (stored 0%)
updating: 15min_total_bytes_raw_no_log/prophet_model/ (stored 0%)
updating: FINAL_BENCHMARK/ (stored 0%)
updating: FINAL_BENCHMARK/final_report.txt (deflated 67%)
updating: FINAL_BENCHMARK/comprehensive_comparison.csv (deflated 50%)
updating: FINAL_BENCHMARK/benchmark_visualizations.png (deflated 16%)
updating: 5min_total_bytes_final/ (stored 0%)
updating: 5min_tota

/kaggle/working/all_output.zip